In [34]:
# Pandas ve Plotly kütüphanelerini içe aktarıyoruz
import pandas as pd # pandas kütüphanesi, veri manipülasyonu ve analizi için kullanılır.
import plotly.io as pio # plotly.io, Plotly grafiklerinin çeşitli ayarlarını yapılandırmak ve kontrol etmek için kullanılır.
import plotly.graph_objects as go # plotly.io, Plotly grafiklerinin çeşitli ayarlarını yapılandırmak ve kontrol etmek için kullanılır.
pio.templates.default = "plotly_white" # Plotly'nin varsayılan şablonunu belirliyoruz
import plotly.express as px  # plotly.express, yüksek seviyeli bir arayüz sağlayarak hızlı ve kolay bir şekilde interaktif grafikler oluşturmanıza yardımcı olur.

# pd.read_csv ile "Apple-Fitness-Data.csv" dosyasından veriyi okuyoruz
data = pd.read_csv("Apple-Fitness-Data.csv")

# head() metodu ile verinin ilk beş satırını görüntülüyoruz
data.head()

,Date,Time,Step Count,Distance,Energy Burned,Flights Climbed,Walking Double Support Percentage,Walking Speed
0,2023-03-21,16:01:23,46,0.02543,14.620,3,0.304,3.060
1,2023-03-21,16:18:37,645,0.40041,14.722,3,0.309,3.852
2,2023-03-21,16:31:38,14,0.00996,14.603,4,0.278,3.996
3,2023-03-21,16:45:37,13,0.00901,14.811,3,0.278,5.040
4,2023-03-21,17:10:30,17,0.00904,15.153,3,0.281,5.184


In [35]:
# Her sütundaki eksik değerlerin sayısını hesaplıyoruz
data.isnull().sum()

# data.isnull() her hücredeki boş değeri kontrol eder ve her hücre için True veya False döndürür.
# data.isnull().sum() her sütundaki boş değerlerin sayısını toplar ve sonuç olarak her sütun için boş değer sayısını verir.

Date                                 0
Time                                 0
Step Count                           0
Distance                             0
Energy Burned                        0
Flights Climbed                      0
Walking Double Support Percentage    0
Walking Speed                        0
dtype: int64

In [36]:
# 1. çizim: Zaman İçinde Adım Sayısı
cizim1 = px.bar(data, x="Time",
                 y= "Step Count",
                 title="Zaman İçinde Adım Sayısı")
cizim1.show()

In [38]:
# 2. çizim: Zaman Göre Kat Edilen Mesafe
cizim2= px.bar(data, x="Time",
                y= "Distance",
                title="Zama Göre kat Edilen Mesafe")
cizim2.show()

In [39]:
# 3. çizim: Zaman İçinde Yakılan Enerji
cizim3= px.bar(data, x="Time",
                y= "Energy Burned",
                title= "Zaman içinde Yakılan Enerji")
cizim3.show()

In [40]:
# 4. çizim: Zaman İçinde Yürüme Hızı
cizim4= px.line(data, x="Time",
                y= "Walking Speed",
                title= "Zaman İçinde Yürüme Hızı")
cizim4.show()

In [41]:
# Günlük ortalama adım sayısını hesaplıyoruz
Gunluk_ortlama_adim_sayisi = data.groupby("Date")["Step Count"].mean().reset_index()

# 5. çizim: Günlük Ortalama Adım Sayısı
cizim5 = px.bar(Gunluk_ortlama_adim_sayisi, x = "Date",
                y = "Step Count",
                title = "Günlük Ortalama Adım Sayısı")
cizim5.update_xaxes(type='category')
cizim5.show()

In [42]:
# Yürüme Verimliliği hesaplanıyor
data["Walking Efficiency"] = data["Distance"] / data["Step Count"]

# 6. çizim: Zaman İçindeki Yürüyüş Verimliliği
cizim6 = px.line(data, x="Time",
                 y="Walking Efficiency",
                 title="Zaman içindeki yürüyğş verimliliği")

cizim6.show()

In [43]:
# Zaman aralıklarını hesaplıyoruz

"""
İlk olarak, "Zaman" sütunundaki saat bilgilerini alarak hangi
zaman dilimine ait olduğunu belirlemek amacıyla "Zaman Aralıkları"
oluşturuyoruz. Bu işlemi pd.to_datetime(data["Time"]).dt.hour
ile yapıyoruz. pd.to_datetime() işlevi ile "Time" sütunu
datetime nesnelerine dönüştürülüp, saat bilgisi alınıyor.

"Zaman Aralıkları" oluştururken, bins parametresini kullanarak
saatleri grupluyoruz. Burada [0, 12, 18, 24] saat aralıkları
belirledik. Bu, 0-12 saat arası "Sabah", 12-18 saat arası "Öğle",
ve 18-24 saat arası "Akşam" olarak gruplandırılmasını sağlar.
labels parametresi ile bu gruplara isimler veriyoruz.
"""
zaman_araliklari = pd.cut(pd.to_datetime(data["Time"]).dt.hour,
                          bins=[0, 12, 18, 24],
                          labels=["Sabah", "Öğle", "Akşam"],
                          right = False)

# "Time Interval" adlı yeni bir sütun ekliyoruz

"""
Oluşturduğumuz "Zaman Aralıkları"nı yeni bir sütun olarak veri
çerçevesine ekliyoruz ve bu sütuna "Time Interval" adını veriyoruz.
Bu sütun, her veri noktasının hangi zaman aralığına ait olduğunu
gösterir.
"""
data["Time Interval"] = zaman_araliklari

# 7. çizim: Zaman Aralıklarına Göre Adım Sayısı ve Yürüme Hızı Değişimleri
cizim7 = px.scatter(data, x="Step Count",
                    y= "Walking Speed",
                    color = "Time Interval",
                    title = "Zaman aralıklarına göre adım sayısı ve yürüme hızı değişimkleri",
                    trendline="ols")

cizim7.show()

In [44]:
# Günlük ortalama metrikleri hesaplıyoruz

"""
data.groupby("Date"): Veri çerçevesini "Date" sütununa göre gruplandırır. Bu işlem, her bir tarih değeri için ayrı bir grup oluşturur.

.mean(): Her grup için belirtilen metriklerin (örneğin, "Step Count", "Distance", "Energy Burned" vb.) günlük ortalama değerini hesaplar. Bu, her tarih için bu metriklerin ortalama değerini içeren yeni bir gruplu veri çerçevesi oluşturur.

.reset_index(): Gruplandırma işlemi sonucunda oluşturulan yeni veri çerçevesinin indeksini sıfırlar ve birinci seviyede sıradan bir sayısal indeks ekler. Bu, sonuç veri çerçevesinin daha düzgün ve kullanışlı bir yapıda olmasını sağlar.
"""
gunluk_ortalama_metrikler = data.groupby("Date").mean().reset_index()

# 8. çizim: Günlük Farklı Metriklerin Ortalamaları Treemap
"""
gunluk_ortalama_metrikler.melt(): Bu işlem, gunluk_ortalama_metrikler veri çerçevesini eritme işlemi olarak bilinir. Y
ani, veri çerçevesinin yapısını değiştirerek, veriyi daha uzun bir formata dönüştürür.

id_vars=["Date"]: "Date" sütununu "id_vars" olarak belirleriz. Bu sütun, yeniden düzenlenen verinin ana
tanımlayıcılarından biridir ve sabit kalır. Yani, her tarih için bu sütundaki değerler korunur.

value_vars=["Step Count", "Distance", "Energy Burned", "Flights Climbed", "Walking Double Support Percentage",
"Walking Speed"]: Bu, dönüştürülen verinin "value_vars" olarak belirlenen sütunları içereceğini belirtir.
Yani, bu sütunlar eritme işlemi sonucunda yeni bir sütun olarak kullanılır ve bu sütunlar altındaki değerler,
esas değerler (adım sayısı, mesafe, enerji yakma, vb.) olarak kullanılır.
"""

gunluk_ortalama_metrikler_melted = gunluk_ortalama_metrikler.melt(id_vars=["Date"],
                                                  value_vars=["Step Count", "Distance",
                                                              "Energy Burned", "Flights Climbed",
                                                              "Walking Double Support Percentage",
                                                              "Walking Speed"])

cizim8 = px.treemap(gunluk_ortalama_metrikler_melted,
                 path=["variable"],
                 values="value",
                 color="variable",
                 hover_data=["value"],
                 title="Günlük Farklı Metriklerin Ortalamaları")
cizim8.show()

<ipython-input-44-ef20b8946a9f>:10: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [45]:
# 9. çizim: Günlük Ortalama Metriklerin Treemap Görselleştirmesi (Adım Sayısı hariç)
metrikleri_gorsellestirme = ["Distance", "Energy Burned", "Flights Climbed",
                             "Walking Double Support Percentage", "Walking Speed"]

gunluk_ortalama_metrikler_melted = gunluk_ortalama_metrikler.melt(id_vars=["Date"], value_vars = metrikleri_gorsellestirme)

cizim9 = px.treemap(gunluk_ortalama_metrikler_melted,
                    path=["variable"],
                    values="value",
                    color="variable",
                    hover_data=["value"],
                    title="Daily Averages for Different Metrics (Excluding Step Count)")
cizim9.show()